<table>
    <tr>
        <td  style="background-color:#ffffff;" width="18%"><img src="..\images\qworld.jpg"  align="left"></td>
        <td style="background-color:#ffffff;" width="*"></td>
        <td  style="background-color:#ffffff;vertical-align:text-top;" width="21%"><img src="..\images\qpoland.jpg" align="right"></td>  
        <td  style="background-color:#ffffff;vertical-align:text-top;" width="19%" ><img src="..\images\qlatvia.jpg" align="right"></td>  
    </tr>
    <tr><td colspan="4" align="right" style="color:#777777;background-color:#ffffff;font-size:12px;">
        prepared by Anastasija Trizna
    </td></tr>
    <tr><td colspan="4" align="right" style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;">
        This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros.
    </td></tr>
</table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $

We will modify our <b>SendState</b> function to introduce some errors. 

<b>NoisyChannel</b> function will help us to consider more real-world QKD implementation case.

Below funtion will introduce 12.5% chance that qubit now has an error:

In [1]:
# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

#Initial source: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb
#Code modified to introduce noise in communication channel

def NoisyChannel(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates, simulate noisy quantum channel, where Pauli errors (X - bit flip; Z - phase flip
        will occur in qc2 and then initializes another circuit qc2 with introduce noise.
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    # Parse the instructions and apply to new circuit
    for instruction in qs:
        if instruction[0] == 'x':
            old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')
    
    #Introducing noise
    for instruction in qs:
        if randrange(7)<1:
            old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr]) #apply bit-flip error
        if randrange(7)<1:
            old_qr = int(instruction[5:-1])
            qc2.z(qreg[old_qr]) #apply phase-flip error

<h3>Task 1: Introducing errors</h3>

Modify to use <b>NoisyChannel</b> and run again code you developed in 'BB84' notebook to verify calculated QBER rate and final key Asja and Balvis now share.

In [2]:
def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

In [3]:
#
# your code is here
#

In [4]:
#Solution
qreg = QuantumRegister(32) # quantum register with 8 qubits
creg = ClassicalRegister(32) # classical register with 8 bits

send=[] #Initial bit string ot send
asja_basis=[] #Register to save information about encoding basis
balvis_basis=[] #Register to save information about decoding basis

#Asja
asja = QuantumCircuit(qreg, creg, name='Asja')

for i in range(32):
    bit = randrange(2)
    send.append(bit)
for i, n in enumerate(send):
    if n==1: asja.x(qreg[i]) # apply x-gate
for i in range(32):
    r=randrange(2) #Asja randomly pick a basis
    if r==0: #if bit is 0, then she encodes in Z basis
        asja_basis.append('Z')
    else: #if bit is 1, then she encodes in X basis
        asja.h(qreg[i])
        asja_basis.append('X')

balvis = QuantumCircuit(qreg, creg, name='Balvis') #Defining Balvis circuit
NoisyChannel(asja, balvis, 'Asja') #Asja sends noisy states to Balvis

#Balvis 
for i in range(32):
    r=randrange(2) #Balvis randomly pick a basis
    if r==0: #if bit is 0, then measures in Z basis
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('Z')
    else: #if bit is 1, then measures in X basis
        balvis.h(qreg[i])
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('X')

job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1)
counts = job.result().get_counts(balvis)
counts = print_outcomes_in_reserve(counts)
received = list(map(int, counts))

#Sifting
asja_key=[] #Asjas register for matching rounds
balvis_key=[] #Balvis register for matching rounds
for j in range(0,len(asja_basis)): #Going through list of bases 
    if asja_basis[j] == balvis_basis[j]: #Comparing
        asja_key.append(send[j])
        balvis_key.append(received[j]) #Keeping key bit if bases matched
    else:
        pass #Discard round if bases mismatched

#QBER
rounds = len(asja_key)//3
errors=0
for i in range(rounds):
    bit_index = randrange(len(asja_key)) 
    tested_bit = asja_key[bit_index]
#    print ("Asja randomly selected bit index - ", bit_index, "- and its value is - ", tested_bit)
    if asja_key[bit_index]!=balvis_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    del asja_key[bit_index] #removing tested bits from key strings
    del balvis_key[bit_index]
QBER=errors/rounds #calculating QBER
        
print("QBER value =", QBER)
print("Asjas secret key =", asja_key)
print("Balvis secret key =", balvis_key)

QBER value = 0.16666666666666666
Asjas secret key = [1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]
Balvis secret key = [1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]
